In [ ]:
# MOD per system
import sys
sys.path.append('C:/Users/emiga/OneDrive/Cal/GWs/code/holodeck') 


# %load ../init.ipy
%reload_ext autoreload
%autoreload 2
from importlib import reload

import os
import sys
import logging
import warnings
import numpy as np
import astropy as ap
import scipy as sp
import scipy.stats
import matplotlib as mpl
import matplotlib.pyplot as plt

import h5py
import tqdm.notebook as tqdm

import kalepy as kale
import kalepy.utils
import kalepy.plot

import holodeck as holo
import holodeck.sam
from holodeck import cosmo, utils, plot
from holodeck.constants import MSOL, PC, YR, MPC, GYR

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
mpl.style.use('default')   # avoid dark backgrounds from dark theme vscode
plt.rcParams.update({'grid.alpha': 0.5})

log = holo.log
log.setLevel(logging.INFO)

# 1 Functions

gws_by_loops

In [ ]:
def gws_by_loops(edges, number, realize, round = False, sum=False, print_test = False):
       
    """ Inefficient way to calculate strain from numbered 
    grid integrated

    Parameters
    ----------
    edges : (4,) list of 1darrays
        A list containing the edges along each dimension.  The four dimensions correspond to
        total mass, mass ratio, redshift, and observer-frame orbital frequency.
        The length of each of the four arrays is M, Q, Z, F.
    number : (M-1, Q-1, Z-1, F-1) ndarray
        The number of binaries in each bin of parameter space.  This is calculated by integrating
        `dnum` over each bin.
    realize : bool or int,
        Specification of how to construct one or more discrete realizations.
        If a `bool` value, then whether or not to construct a realization.
        If a `int` value, then how many discrete realizations to construct.
    round : bool
        Specification of whether to discretize the sample if realize is False, 
        by rounding number of binaries in each bin to integers. This has no impact 
        if realize is true.
    sum : bool
        Whether or not to sum the strain at a given frequency over all bins.
    print_test : bool
        Whether or not to print variables as they are calculated, for dev purposes.


    Returns
    -------
    hc : ndarray
        Characteristic strain of the GWB.
        The shape depends on whether realize is an integer or not
        realize = True or False: shape is (M-1, Q-1, Z-1, F-1)
        realize = R: shape is  (M-1, Q-1, Z-1, F-1, R)

    """
    if(print_test):
        print('INPUTS: edges:', len(edges), '\n', edges, 
        '\nINPUTS:number:', number.shape, '\n', number,'\n')

    # Frequency bin midpoints
    foo = edges[-1]                   #: should be observer-frame orbital-frequencies
    df = np.diff(foo)                 #: frequency bin widths
    fc = kale.utils.midpoints(foo)    #: use frequency-bin centers for strain (more accurate!)

    # All other bin midpoints
    mt = kale.utils.midpoints(edges[0]) #: total mass
    mr = kale.utils.midpoints(edges[1]) #: mass ratio
    rz = kale.utils.midpoints(edges[2]) #: redshift

    
    # make strain grid
    hc_grid = np.empty_like(number)

    # for r realizations, make a grid with shape 
    # m, q, z, f, r
    if(utils.isinteger(realize)):
        newshape = hc_grid.shape + (realize,)
        if(print_test):
            print('newshape:', newshape)
        realized_grid = np.empty(newshape)

    # get strain for each bin
    for m_idx in range(len(mt)):
        for q_idx in range(len(mr)):
            cmass = holo.utils.chirp_mass_mtmr(mt[m_idx], mr[q_idx])
            for z_idx in range(len(rz)):
                cdist = holo.cosmo.comoving_distance(rz[z_idx]).cgs.value
                
                # print M, q, z, M_c, d_c
                if(print_test):
                    print('BIN mt=%.2e, mr=%.2e, rz=%.2e' %
                        (mt[m_idx], mr[q_idx], rz[z_idx]))
                    print('\t m_c = %.2e, d_c = %.2e' 
                        % (cmass, cdist))

                for f_idx in range(len(fc)):
                    rfreq = holo.utils.frst_from_fobs(fc[f_idx], rz[z_idx])
                    hs_mqzf = utils.gw_strain_source(cmass, cdist, rfreq)
                    hc_dlnf = hs_mqzf**2 * (fc[f_idx]/df[f_idx])
                    
                    if(realize == False):
                        if(round == True):
                            hc_grid[m_idx, q_idx, z_idx, f_idx] = np.sqrt(hc_dlnf 
                                        * np.floor(number[m_idx, q_idx, z_idx, f_idx]).astype(int))
                        else:
                            hc_grid[m_idx, q_idx, z_idx, f_idx] = np.sqrt(hc_dlnf 
                                        * number[m_idx, q_idx, z_idx, f_idx])
                        
                    elif(realize == True):
                        hc_grid[m_idx, q_idx, z_idx, f_idx] = np.sqrt(hc_dlnf 
                                        *np.random.poisson(number[m_idx, q_idx, z_idx, f_idx]))
                    elif(utils.isinteger(realize)):
                        for r_idx in range(realize):
                            realized_grid[m_idx, q_idx, z_idx, f_idx, r_idx] = \
                                np.sqrt(hc_dlnf 
                                        *np.random.poisson(number[m_idx, q_idx, z_idx, f_idx]))
                        hc_grid = realized_grid                
                    else:
                        print("`realize` ({}) must be one of {{True, False, integer}}!"\
                            .format(realize))

                     # print fr, hs, hc^2/dlnf, number, hc
                    if(print_test):
                        print('\t\tfr = %.2fnHz, h_s = %.2e, h_c^2/dlnf = %.2e' 
                            % (rfreq*10**9, hs_mqzf, hc_dlnf))
                        print('\t\tnumber: %.2e' % number[m_idx, q_idx, z_idx, f_idx])
                        print('\t\thc = %.2e' % hc_grid[m_idx, q_idx, z_idx, f_idx])
    if(sum):
        # sum over all bins at a given frequency and realization
        hc_grid = np.sqrt(np.sum(hc_grid**2, axis=(0, 1, 2)))

    
    return hc_grid


cleaned up loops

In [ ]:
def ss_gws_by_loops(edges, number, realize=False, round=True, sum=True, 
                    ss=True, print_test = False):
       
    """ Inefficient way to calculate strain from numbered 
    grid integrated

    Parameters
    ----------
    edges : (4,) list of 1darrays
        A list containing the edges along each dimension.  The four dimensions correspond to
        total mass, mass ratio, redshift, and observer-frame orbital frequency.
        The length of each of the four arrays is M, Q, Z, F.
    number : (M-1, Q-1, Z-1, F-1) ndarray
        The number of binaries in each bin of parameter space.  This is calculated by integrating
        `dnum` over each bin.
    realize : bool or int,
        Specification of how to construct one or more discrete realizations.
        If a `bool` value, then whether or not to construct a realization.
        If a `int` value, then how many discrete realizations to construct.
    round : bool
        Specification of whether to discretize the sample if realize is False, 
        by rounding number of binaries in each bin to integers. This has no impact 
        if realize is true.
    sum : bool
        Whether or not to sum the strain at a given frequency over all bins.
    ss : bool 
        Whether or not to separate the loudest single source in each frequency bin.
        NOTE: would be good to update this to allow for multiple foreground sources.
    print_test : bool
        Whether or not to print variable as they are calculated, for dev purposes.


    Returns
    -------
    hc : ndarray
        Characteristic strain of the GWB.
        The shape depends on whether realize is an integer or not
        realize = True or False, sum = False: shape is (M-1, Q-1, Z-1, F-1)
        realize = True or False, sum = True: shape is (F-1)
        realize = R, sum = False: shape is  (M-1, Q-1, Z-1, F-1, R)
        realize = R, sum = True: shape is  (F-1, R)
    louds : (F-1, 3) ndarray or None
        M, q, and z of the loudest single source in each frequency bin.
        None if loud = False


    """
    if(print_test):
        print('INPUTS: edges:', len(edges), '\n', edges, 
        '\nINPUTS:number:', number.shape, '\n', number,'\n')

    # Frequency bin midpoints
    foo = edges[-1]                   #: should be observer-frame orbital-frequencies
    df = np.diff(foo)                 #: frequency bin widths
    fc = kale.utils.midpoints(foo)    #: use frequency-bin centers for strain (more accurate!)

    # All other bin midpoints
    mt = kale.utils.midpoints(edges[0]) #: total mass
    mr = kale.utils.midpoints(edges[1]) #: mass ratio
    rz = kale.utils.midpoints(edges[2]) #: redshift

    
    # GW background characteristic strain
    hc_bg = np.empty_like(number)
    new_num = np.copy(number)

    # for single sources, make a grid with shape
    # (f, 3)
    if(ss == True):
        # params of loudest bin with number>=1
        # shape (f,3) for 3 params
        sspar = np.empty((len(fc), 3)) 
        # param indices of loudest bin with number>=1
        # shape (f,3) for 3 params
        ssidx = np.empty((len(fc), 3)) 
        # max hs at each frequency
        maxhs = np.zeros(len(fc))
        # (max)  single source characteristic strain at each frequency
        hc_ss = np.zeros(len(fc))
    else: 
        sspar = None
        ssidx = None
        maxhs = None
        hc_ss = None

    # # not worrying about realization implementation yet
    # # for r realizations, make a grid with shape 
    # # m, q, z, f, r
    # if(utils.isinteger(realize)):
    #     newshape = hc_grid.shape + (realize,)
    #     if(print_test):
    #         print('newshape:', newshape)
    #     realized_grid = np.empty(newshape)


    # --------------- Single Sources ------------------
    # 1) Identify the loudest (max hs) single source in a bin with N>=1
    # 2) Record the parameters, parameter indices, and strain
    #  of that single source
    # 3) Subtract 1 from the number in that source's bin, 
    # 4) Calculate single source characteristic strain (hc)
    # 5) Calculate the background with the new number 
    
    if(ss == True):
        for m_idx in range(len(mt)):
            for q_idx in range(len(mr)):
                cmass = holo.utils.chirp_mass_mtmr(mt[m_idx], mr[q_idx])
                for z_idx in range(len(rz)):
                    cdist = holo.cosmo.comoving_distance(rz[z_idx]).cgs.value
                    
                    # print M, q, z, M_c, d_c
                    if(print_test):
                        print('BIN mt=%.2e, mr=%.2e, rz=%.2e' %
                            (mt[m_idx], mr[q_idx], rz[z_idx]))
                        print('\t m_c = %.2e, d_c = %.2e' 
                            % (cmass, cdist))

                    # check if loudest source in any bin
                    for f_idx in range(len(fc)):
                        rfreq = holo.utils.frst_from_fobs(fc[f_idx], rz[z_idx])
                        # hs of a source in that bin
                        hs_mqzf = utils.gw_strain_source(cmass, cdist, rfreq)
                        
                        # 1) IF LOUDEST
                        # check if loudest hs at that 
                        # frequency and contains binaries
                        if(hs_mqzf>maxhs[f_idx] and 
                           number[m_idx, q_idx, z_idx, f_idx]>=1):
                            print('number:', number[m_idx, q_idx, z_idx, f_idx])

                            # 2) If so, RECORD:
                            # parameters M, q, z
                            sspar[f_idx] = np.array([mt[m_idx], mr[q_idx],
                                                     rz[z_idx]])
                            # parameter indices
                            ssidx[f_idx] = np.array([m_idx, q_idx, z_idx])
                            # new max strain
                            maxhs[f_idx] = hs_mqzf

                    # print fr, hs, hc^2/dlnf, number, hc
                    # if(print_test):
                    #     print('\tfr = %.2fnHz, h_s = %.2e'
                    #         % (rfreq*10**9, hs_mqzf))
                    #     print('\t\tnumber: %.2e' % 
                    #           number[m_idx, q_idx, z_idx, f_idx])
                    #     if(ss):
                    #         print('\t\thsmax = %.2e, max indices = (%d,%d,%d)' 
                    #             % (maxhs[f_idx], ssidx[f_idx,0], 
                    #                ssidx[f_idx,1], ssidx[f_idx,2]))

        # 3) SUBTRACT 1 
        # from bin with loudest source at each frequency
        # can do this using the index of loudest, ssidx
        # recall ssidx has shape [3, F]
        # and = [(m_idx,q_idx,z_idx), fc],
        for f_idx in range(len(fc)):
            new_num[int(ssidx[f_idx,0]), int(ssidx[f_idx,1]), int(ssidx[f_idx,2]), 
                    f_idx] = number[int(ssidx[f_idx,0]), int(ssidx[f_idx,1]),
                                     int(ssidx[f_idx,2]), f_idx] - 1 
            # print('ssidx=', int(ssidx[f_idx,0]), int(ssidx[f_idx,1]), int(ssidx[f_idx,2]), f_idx)
            # print(new_num.shape)
            # print(new_num[int(ssidx[f_idx,0]), int(ssidx[f_idx,1]), int(ssidx[f_idx,2]), f_idx])

            # 4) CALCULATE 
            # single source characteristic strain
            hc_ss[f_idx] = np.sqrt(maxhs[f_idx]**2 * (fc[f_idx]/df[f_idx]))

        # CHECK: no numbers should be <0 after subtraction
        if(np.any(new_num<0)): 
            error_index = np.where(new_num<0)
            print('number<0 found at (M,q,z,f) =', error_index)         

    # 5)
    # ----------------- Calculate Background Strains --------------------
    # then we can go back in and calculate characteristic strains
    # NOTE: could make this faster by saving rfreq and hs values from above
    # instead of recalculating
    for m_idx in range(len(mt)):
        for q_idx in range(len(mr)):
            cmass = holo.utils.chirp_mass_mtmr(mt[m_idx], mr[q_idx])
            for z_idx in range(len(rz)):
                cdist = holo.cosmo.comoving_distance(rz[z_idx]).cgs.value
                for f_idx in range(len(fc)):
                    rfreq = holo.utils.frst_from_fobs(fc[f_idx], rz[z_idx])
                    hs_mqzf = utils.gw_strain_source(cmass, cdist, rfreq)
                    hc_dlnf = hs_mqzf**2 * (fc[f_idx]/df[f_idx])
                    if(realize == False):
                        if(round == True):
                            new_num[m_idx, q_idx, z_idx, f_idx] = np.floor(new_num[m_idx, q_idx, z_idx, f_idx]).astype(int)
                            hc_bg[m_idx, q_idx, z_idx, f_idx] = np.sqrt(hc_dlnf 
                                        * new_num[m_idx, q_idx, z_idx, f_idx])
                        else:
                            hc_bg[m_idx, q_idx, z_idx, f_idx] = np.sqrt(hc_dlnf 
                                        * new_num[m_idx, q_idx, z_idx, f_idx])
                    else: 
                        print('realize not implemented yet') 
                        return 0   
                    # elif(realize == True):
                    #     hc_grid[m_idx, q_idx, z_idx, f_idx] = np.sqrt(hc_dlnf 
                    #                     *np.random.poisson(new_num[m_idx, q_idx, z_idx, f_idx]))
                    # elif(utils.isinteger(realize)):
                    #     for r_idx in range(realize):
                    #         realized_grid[m_idx, q_idx, z_idx, f_idx, r_idx] = \
                    #             np.sqrt(hc_dlnf 
                    #                     *np.random.poisson(new_num[m_idx, q_idx, z_idx, f_idx]))
                    #     hc_grid = realized_grid                
                    # else:
                    #     print("`realize` ({}) must be one of {{True, False, integer}}!"\
                    #         .format(realize))

                    if(print_test):
                        print('\tfr = %.2fnHz, h_s = %.2e, h_c^2/dlnf = %.2e' 
                            % (rfreq*10**9, hs_mqzf, hc_dlnf))
                        print('\t\tnumber: %.2e' % new_num[m_idx, q_idx, z_idx, f_idx])
                        print('\t\thc = %.2e' % hc_bg[m_idx, q_idx, z_idx, f_idx])
                        if(ss == True):
                            print('\t\t loudest?', 
                                np.all((ssidx[f_idx] == np.array([m_idx, q_idx, z_idx]))))
    if(ss and print_test):
        print('----loudest bins:')
        for f_idx in range(len(fc)):
            print('\t M=%.2e, q=%.2e, z=%.2e, f=%.2e' 
                    % (sspar[f_idx,0], sspar[f_idx,1], sspar[f_idx,2], fc[f_idx]))
            # if we use this instead, we don't need sspar
            # print('\t M=%.2e, q=%.2e, z=%.2e, f=%.2e' 
            #       % (mt[ssidx[f_idx,0]], mr[ssidx[f_idx,1]], rz[ssidx[f_idx,2]]))

     
    if(sum):
        # sum over all bins at a given frequency and realization
        hc_bg = np.sqrt(np.sum(hc_bg**2, axis=(0, 1, 2)))

    
    return hc_bg, hc_ss, sspar, ssidx, maxhs, new_num


rounding first loops

In [ ]:
def ss_gws_by_loops_2(edges, number, realize=False, round=True, sum=True, 
                    ss=True, print_test = False):
       
    """ Inefficient way to calculate strain from numbered 
    grid integrated

    Parameters
    ----------
    edges : (4,) list of 1darrays
        A list containing the edges along each dimension.  The four dimensions correspond to
        total mass, mass ratio, redshift, and observer-frame orbital frequency.
        The length of each of the four arrays is M, Q, Z, F.
    number : (M-1, Q-1, Z-1, F-1) ndarray
        The number of binaries in each bin of parameter space.  This is calculated by integrating
        `dnum` over each bin.
    realize : bool or int,
        Specification of how to construct one or more discrete realizations.
        If a `bool` value, then whether or not to construct a realization.
        If a `int` value, then how many discrete realizations to construct.
    round : bool
        Specification of whether to discretize the sample if realize is False, 
        by rounding number of binaries in each bin to integers. This has no impact 
        if realize is true.
    sum : bool
        Whether or not to sum the strain at a given frequency over all bins.
    ss : bool 
        Whether or not to separate the loudest single source in each frequency bin.
        NOTE: would be good to update this to allow for multiple foreground sources.
    print_test : bool
        Whether or not to print variable as they are calculated, for dev purposes.


    Returns
    -------
    hc : ndarray
        Characteristic strain of the GWB.
        The shape depends on whether realize is an integer or not
        realize = True or False, sum = False: shape is (M-1, Q-1, Z-1, F-1)
        realize = True or False, sum = True: shape is (F-1)
        realize = R, sum = False: shape is  (M-1, Q-1, Z-1, F-1, R)
        realize = R, sum = True: shape is  (F-1, R)
    louds : (F-1, 3) ndarray or None
        M, q, and z of the loudest single source in each frequency bin.
        None if loud = False


    """
    if(print_test):
        print('INPUTS: edges:', len(edges), '\n', edges, 
        '\nINPUTS:number:', number.shape, '\n', number,'\n')

    # Frequency bin midpoints
    foo = edges[-1]                   #: should be observer-frame orbital-frequencies
    df = np.diff(foo)                 #: frequency bin widths
    fc = kale.utils.midpoints(foo)    #: use frequency-bin centers for strain (more accurate!)

    # All other bin midpoints
    mt = kale.utils.midpoints(edges[0]) #: total mass
    mr = kale.utils.midpoints(edges[1]) #: mass ratio
    rz = kale.utils.midpoints(edges[2]) #: redshift

    
    # GW background characteristic strain
    hc_bg = np.empty_like(number)

    # new number array
    if(round == True):
        new_num = np.copy(np.floor(number).astype(int))
        print(new_num[np.where(new_num%1 !=0)])
    else:
        new_num = np.copy(number)

    # for single sources, make a grid with shape
    # (f, 3)
    if(ss == True):
        # params of loudest bin with number>=1
        # shape (f,3) for 3 params
        sspar = np.empty((len(fc), 3)) 
        # param indices of loudest bin with number>=1
        # shape (f,3) for 3 params
        ssidx = np.empty((len(fc), 3)) 
        # max hs at each frequency
        maxhs = np.zeros(len(fc))
        # (max)  single source characteristic strain at each frequency
        hc_ss = np.zeros(len(fc))
    else: 
        sspar = None
        ssidx = None
        maxhs = None
        hc_ss = None

    # # not worrying about realization implementation yet
    # # for r realizations, make a grid with shape 
    # # m, q, z, f, r
    # if(utils.isinteger(realize)):
    #     newshape = hc_grid.shape + (realize,)
    #     if(print_test):
    #         print('newshape:', newshape)
    #     realized_grid = np.empty(newshape)


    # --------------- Single Sources ------------------
    # 1) Identify the loudest (max hs) single source in a bin with N>=1
    # 2) Record the parameters, parameter indices, and strain
    #  of that single source
    # 3) Subtract 1 from the number in that source's bin, 
    # 4) Calculate single source characteristic strain (hc)
    # 5) Calculate the background with the new number 
    
    if(ss == True):
        for m_idx in range(len(mt)):
            for q_idx in range(len(mr)):
                cmass = holo.utils.chirp_mass_mtmr(mt[m_idx], mr[q_idx])
                for z_idx in range(len(rz)):
                    cdist = holo.cosmo.comoving_distance(rz[z_idx]).cgs.value
                    
                    # print M, q, z, M_c, d_c
                    if(print_test):
                        print('BIN mt=%.2e, mr=%.2e, rz=%.2e' %
                            (mt[m_idx], mr[q_idx], rz[z_idx]))
                        print('\t m_c = %.2e, d_c = %.2e' 
                            % (cmass, cdist))

                    # check if loudest source in any bin
                    for f_idx in range(len(fc)):
                        rfreq = holo.utils.frst_from_fobs(fc[f_idx], rz[z_idx])
                        # hs of a source in that bin
                        hs_mqzf = utils.gw_strain_source(cmass, cdist, rfreq)
                        
                        # 1) IF LOUDEST
                        # check if loudest hs at that 
                        # frequency and contains binaries
                        if(hs_mqzf>maxhs[f_idx] and 
                           new_num[m_idx, q_idx, z_idx, f_idx]>0):
                            if(new_num[m_idx, q_idx, z_idx, f_idx]<1):
                                print('number<1 used', new_num[m_idx, q_idx, z_idx, f_idx])  #DELETE
                            # 2) If so, RECORD:
                            # parameters M, q, z
                            sspar[f_idx] = np.array([mt[m_idx], mr[q_idx],
                                                     rz[z_idx]])
                            # parameter indices
                            ssidx[f_idx] = np.array([m_idx, q_idx, z_idx])
                            # new max strain
                            maxhs[f_idx] = hs_mqzf

                    # print fr, hs, hc^2/dlnf, number, hc
                    # if(print_test):
                    #     print('\tfr = %.2fnHz, h_s = %.2e'
                    #         % (rfreq*10**9, hs_mqzf))
                    #     print('\t\tnumber: %.2e' % 
                    #           number[m_idx, q_idx, z_idx, f_idx])
                    #     if(ss):
                    #         print('\t\thsmax = %.2e, max indices = (%d,%d,%d)' 
                    #             % (maxhs[f_idx], ssidx[f_idx,0], 
                    #                ssidx[f_idx,1], ssidx[f_idx,2]))

        # 3) SUBTRACT 1 
        # from bin with loudest source at each frequency
        # can do this using the index of loudest, ssidx
        # recall ssidx has shape [3, F]
        # and = [(m_idx,q_idx,z_idx), fc],
        for f_idx in range(len(fc)):
            new_num[int(ssidx[f_idx,0]), int(ssidx[f_idx,1]), int(ssidx[f_idx,2]), 
                    f_idx] -= 1 
            # print('ssidx=', int(ssidx[f_idx,0]), int(ssidx[f_idx,1]), int(ssidx[f_idx,2]), f_idx)
            # print(new_num.shape)
            # print(new_num[int(ssidx[f_idx,0]), int(ssidx[f_idx,1]), int(ssidx[f_idx,2]), f_idx])

            # 4) CALCULATE 
            # single source characteristic strain
            hc_ss[f_idx] = np.sqrt(maxhs[f_idx]**2 * (fc[f_idx]/df[f_idx]))
        
        # CHECK no numbers should be <0 
        if(np.any(new_num<0)): 
            error_index = np.where(new_num<0)
            print('number<0 found at (M,q,z,f) =', error_index)         
        

    # 5)
    # ----------------- Calculate Background Strains --------------------
    # then we can go back in and calculate characteristic strains
    # NOTE: could make this faster by saving rfreq and hs values from above
    # instead of recalculating
    for m_idx in range(len(mt)):
        for q_idx in range(len(mr)):
            cmass = holo.utils.chirp_mass_mtmr(mt[m_idx], mr[q_idx])
            for z_idx in range(len(rz)):
                cdist = holo.cosmo.comoving_distance(rz[z_idx]).cgs.value
                for f_idx in range(len(fc)):
                    rfreq = holo.utils.frst_from_fobs(fc[f_idx], rz[z_idx])
                    hs_mqzf = utils.gw_strain_source(cmass, cdist, rfreq)
                    hc_dlnf = hs_mqzf**2 * (fc[f_idx]/df[f_idx])
                    if(realize == False):
                        hc_bg[m_idx, q_idx, z_idx, f_idx] = np.sqrt(hc_dlnf 
                                        * new_num[m_idx, q_idx, z_idx, f_idx])
                    else: 
                        print('realize not implemented yet') 
                        return 0   
                    # elif(realize == True):
                    #     hc_grid[m_idx, q_idx, z_idx, f_idx] = np.sqrt(hc_dlnf 
                    #                     *np.random.poisson(new_num[m_idx, q_idx, z_idx, f_idx]))
                    # elif(utils.isinteger(realize)):
                    #     for r_idx in range(realize):
                    #         realized_grid[m_idx, q_idx, z_idx, f_idx, r_idx] = \
                    #             np.sqrt(hc_dlnf 
                    #                     *np.random.poisson(new_num[m_idx, q_idx, z_idx, f_idx]))
                    #     hc_grid = realized_grid                
                    # else:
                    #     print("`realize` ({}) must be one of {{True, False, integer}}!"\
                    #         .format(realize))

                    if(print_test):
                        print('\tfr = %.2fnHz, h_s = %.2e, h_c^2/dlnf = %.2e' 
                            % (rfreq*10**9, hs_mqzf, hc_dlnf))
                        print('\t\tnumber: %.2e' % new_num[m_idx, q_idx, z_idx, f_idx])
                        print('\t\thc = %.2e' % hc_bg[m_idx, q_idx, z_idx, f_idx])
                        if(ss == True):
                            print('\t\t loudest?', 
                                np.all((ssidx[f_idx] == np.array([m_idx, q_idx, z_idx]))))
    if(ss and print_test):
        print('----loudest bins:')
        for f_idx in range(len(fc)):
            print('\t M=%.2e, q=%.2e, z=%.2e, f=%.2e' 
                    % (sspar[f_idx,0], sspar[f_idx,1], sspar[f_idx,2], fc[f_idx]))
            # if we use this instead, we don't need sspar
            # print('\t M=%.2e, q=%.2e, z=%.2e, f=%.2e' 
            #       % (mt[ssidx[f_idx,0]], mr[ssidx[f_idx,1]], rz[ssidx[f_idx,2]]))

     
    if(sum):
        # sum over all bins at a given frequency and realization
        hc_bg = np.sqrt(np.sum(hc_bg**2, axis=(0, 1, 2)))

    
    return hc_bg, hc_ss, sspar, ssidx, maxhs, new_num


In [ ]:
def gws_by_ndars(edges, number, realize, round = True, sum = True, 
                 ss = True, print_test = False):
    """ More efficient way to calculate strain from numbered 
    grid integrated

    Parameters
    ----------
    edges : (4,) list of 1darrays
        A list containing the edges along each dimension.  The four dimensions 
        correspond to total mass, mass ratio, redshift, and observer-frame orbital 
        frequency. The length of each of the four arrays is M, Q, Z, F.
    number : (M-1, Q-1, Z-1, F-1) ndarray
        The number of binaries in each bin of parameter space.  This is calculated 
        by integrating `dnum` over each bin.
    realize : bool or int
        Specification of how to construct one or more discrete realizations.
        If a `bool` value, then whether or not to construct a realization.
        If an `int` value, then how many discrete realizations to construct.
    round : bool
        Specification of whether to discretize the sample if realize is False, 
        by rounding number of binaries in each bin to integers. This has no impact 
        if realize is true.
    sum : bool
        Whether or not to sum the strain at a given frequency over all bins.
    print_test : bool
        Whether or not to print variable as they are calculated, for dev purposes.


    Returns
    -------
    hc : ndarray
        Characteristic strain of the GWB.
        The shape depends on whether realize is an integer or not
        realize = True or False, sum = False: shape is (M-1, Q-1, Z-1, F-1)
        realize = True or False, sum = True: shape is (F-1)
        realize = R, sum = False: shape is  (M-1, Q-1, Z-1, F-1, R)
        realize = R, sum = True: shape is  (F-1, R)

    """

    if(print_test):
        print('INPUTS: edges:', len(edges), # '\n', edges, 
        '\nINPUTS:number:', number.shape, '\n', number,'\n')

    # Frequency bin midpoints
    foo = edges[-1]                   #: should be observer-frame orbital-frequencies
    df = np.diff(foo)                 #: frequency bin widths
    fc = kale.utils.midpoints(foo)    #: use frequency-bin centers for strain (more accurate!)

    # All other bin midpoints
    mt = kale.utils.midpoints(edges[0]) #: total mass
    mr = kale.utils.midpoints(edges[1]) #: mass ratio
    rz = kale.utils.midpoints(edges[2]) #: redshift


    # --- Chirp Masses ---
    # to get chirp mass in shape (M-1, Q-1) we need 
    # mt in shape (M-1, 1) 
    # mr in shape (1, Q-1)
    cmass = utils.chirp_mass_mtmr(mt[:,np.newaxis], mr[np.newaxis,:])
    if(print_test):
        print('cmass:', cmass.shape, '\n', cmass)

    # --- Comoving Distances ---
    # to get cdist in shape (Z-1) we need
    # rz in shape (Z-1)
    cdist = holo.cosmo.comoving_distance(rz).cgs.value
    if(print_test):
        print('cdist:', cdist.shape, '\n', cdist)

    # --- Rest Frame Frequencies ---
    # to get rest freqs in shape (Z-1, F-1) we need 
    # rz in shape (Z-1, 1) 
    # fc in shape (1, F-1)
    rfreq = holo.utils.frst_from_fobs(fc[np.newaxis,:], rz[:,np.newaxis])
    if(print_test):
        print('rfreq:', rfreq.shape, '\n', rfreq)

    # --- Source Strain Amplitude ---
    # to get hs amplitude in shape (M-1, Q-1, Z-1, F-1) we need
    # cmass in shape (M-1, Q-1, 1, 1) from (M-1, Q-1)
    # cdist in shape (1, 1, Z-1, 1) from (Z-1)
    # rfreq in shape (1, 1, Z-1, F-1) from (Z-1, F-1)
    hsamp = utils.gw_strain_source(cmass[:,:,np.newaxis,np.newaxis],
                                   cdist[np.newaxis,np.newaxis,:,np.newaxis],
                                   rfreq[np.newaxis,np.newaxis,:,:])
    if(print_test):
        print('hsamp', hsamp.shape, '\n', hsamp)


    # --------------- Single Sources ------------------
    # 1) Identify the loudest (max hs) single source in a bin with N>=1
    # 2) Record the parameters, parameter indices, and strain
    #  of that single source
    # 3) Subtract 1 from the number in that source's bin, 
    # 4) Calculate single source characteristic strain (hc)
    # 5) Calculate the background with the new number




    # 1) Identify loudest single 

# GW background characteristic strain
    hc_bg = np.empty_like(number)
    new_num = np.copy(number)

    # for single sources, make a grid with shape
    # (f, 3)
    if(ss == True):
        # params of loudest bin with number>=1
        # shape (f,3) for 3 params
        sspar = np.empty((len(fc), 3)) 
        # param indices of loudest bin with number>=1
        # shape (f,3) for 3 params
        ssidx = np.empty((len(fc), 3)) 
        # max hs at each frequency
        maxhs = np.zeros(len(fc))
        # (max)  single source characteristic strain at each frequency
        hc_ss = np.zeros(len(fc))
    else: 
        sspar = None
        ssidx = None
        maxhs = None
        hc_ss = None







    # --- Characteristic Strain Squared ---
    # to get characteristic strain in shape (M-1, Q-1, Z-1, F-1) we need
    # hsamp in shape (M-1, Q-1, Z-1, F-1)
    # fc in shape (1, 1, 1, F-1)
    hchar = hsamp**2 * (fc[np.newaxis, np.newaxis, np.newaxis,:]
                        /df[np.newaxis, np.newaxis, np.newaxis,:])

    # Sample:
    if(realize == False):
        # without sampling, want strain in shape (M-1, Q-1, Z-1, F-1)
        if(round): 
            # discretize by rounding number down to nearest integer 
            hchar *= np.floor(number).astype(int) 
        else: 
            # keep non-integer values
            hchar *= number

    if(realize == True):
        # with a single sample, want strain in shape (M-1, Q-1, Z-1, F-1)
        hchar *= np.random.poisson(number)

    if(utils.isinteger(realize)):
        # with R realizations, 
        # to get strain in shape (M-1, Q-1, Z-1, F-1, R) we need
        # hchar in shape(M-1, Q-1, Z-1, F-1, 1)
        # Poisson sample in shape (1, 1, 1, 1, R)
        npois = np.random.poisson(number[...,np.newaxis], size = (number.shape + (realize,)))
        if(print_test):
            print('npois', npois.shape)
        hchar = hchar[...,np.newaxis] * npois


    if(print_test):
        print('hchar', hchar.shape, '\n', hchar)


    if(sum):
        # sum over all bins at a given frequency and realization
        hchar = np.sum(hchar, axis=(0, 1, 2))
        # NOTE I should check what big O time this is,  not sure
        if(print_test):
            print('hchar summed', hchar.shape, '\n', hchar)

    return np.sqrt(hchar)

some edits for loud_gws:
* no longer makes sense to have a sum=False option


##### ss ndars

In [ ]:
def ss_gws_by_ndars(edges, number, realize, round = False, sum = True, 
                    ss = True, print_test = False):
       
    """ More efficient way to calculate strain from numbered 
    grid integrated

    Parameters
    ----------
    edges : (4,) list of 1darrays
        A list containing the edges along each dimension.  The four dimensions 
        correspond to total mass, mass ratio, redshift, and observer-frame orbital 
        frequency. The length of each of the four arrays is M, Q, Z, F.
    number : (M-1, Q-1, Z-1, F-1) ndarray
        The number of binaries in each bin of parameter space.  This is calculated 
        by integrating `dnum` over each bin.
    realize : bool or int
        Specification of how to construct one or more discrete realizations.
        If a `bool` value, then whether or not to construct a realization.
        If an `int` value, then how many discrete realizations to construct.
    round : bool
        Specification of whether to discretize the sample if realize is False, 
        by rounding number of binaries in each bin to integers. This has no impact 
        if realize is true.
    sum : bool
        Whether or not to sum the strain at a given frequency over all bins.
    loud : bool 
        Whether or not to separate the loudest single source in each frequency bin.
        NOTE: would be good to update this to allow for multiple foreground sources.
    print_test : bool
        Whether or not to print variable as they are calculated, for dev purposes.


    Returns
    -------
    hchar : ndarray
        Characteristic strain of the GWB.
        The shape depends on whether realize is an integer or not
        realize = True or False, sum = False: shape is (M-1, Q-1, Z-1, F-1)
        realize = True or False, sum = True: shape is (F-1)
        realize = R, sum = False: shape is  (M-1, Q-1, Z-1, F-1, R)
        realize = R, sum = True: shape is  (F-1, R)
    louds : (3,F-1) ndarray or None
        Parameters of the loudest single source in each frequency bin.
        None if loud = False

    """

    if(print_test):
        print('INPUTS: edges:', len(edges), # '\n', edges, 
        '\nINPUTS:number:', number.shape, '\n', number,'\n')

    # Frequency bin midpoints
    foo = edges[-1]                   #: should be observer-frame orbital-frequencies
    df = np.diff(foo)                 #: frequency bin widths
    fc = kale.utils.midpoints(foo)    #: use frequency-bin centers for strain (more accurate!)

    # All other bin midpoints
    mt = kale.utils.midpoints(edges[0]) #: total mass
    mr = kale.utils.midpoints(edges[1]) #: mass ratio
    rz = kale.utils.midpoints(edges[2]) #: redshift


    # --- Chirp Masses ---
    # to get chirp mass in shape (M-1, Q-1) we need 
    # mt in shape (M-1, 1) 
    # mr in shape (1, Q-1)
    cmass = utils.chirp_mass_mtmr(mt[:,np.newaxis], mr[np.newaxis,:])
    if(print_test):
        print('cmass:', cmass.shape, '\n', cmass)

    # --- Comoving Distances ---
    # to get cdist in shape (Z-1) we need
    # rz in shape (Z-1)
    cdist = holo.cosmo.comoving_distance(rz).cgs.value
    if(print_test):
        print('cdist:', cdist.shape, '\n', cdist)

    # --- Rest Frame Frequencies ---
    # to get rest freqs in shape (Z-1, F-1) we need 
    # rz in shape (Z-1, 1) 
    # fc in shape (1, F-1)
    rfreq = holo.utils.frst_from_fobs(fc[np.newaxis,:], rz[:,np.newaxis])
    if(print_test):
        print('rfreq:', rfreq.shape, '\n', rfreq)

    # --- Source Strain Amplitude ---
    # to get hs amplitude in shape (M-1, Q-1, Z-1, F-1) we need
    # cmass in shape (M-1, Q-1, 1, 1) from (M-1, Q-1)
    # cdist in shape (1, 1, Z-1, 1) from (Z-1)
    # rfreq in shape (1, 1, Z-1, F-1) from (Z-1, F-1)
    hsamp = utils.gw_strain_source(cmass[:,:,np.newaxis,np.newaxis],
                                   cdist[np.newaxis,np.newaxis,:,np.newaxis],
                                   rfreq[np.newaxis,np.newaxis,:,:])
    if(print_test):
        print('hsamp', hsamp.shape, '\n', hsamp)

    if(loud == True):
        louds = np.empty((3,len(fc)))

    # --- Characteristic Strain Squared ---
    # to get characteristic strain in shape (M-1, Q-1, Z-1, F-1) we need
    # hsamp in shape (M-1, Q-1, Z-1, F-1)
    # fc in shape (1, 1, 1, F-1)
    hchar = hsamp**2 * (fc[np.newaxis, np.newaxis, np.newaxis,:]
                        /df[np.newaxis, np.newaxis, np.newaxis,:])

    # Sample:
    if(realize == False):
        # without sampling, want strain in shape (M-1, Q-1, Z-1, F-1)
        if(round): 
            # discretize by rounding number down to nearest integer 
            hchar *= np.floor(number).astype(int) 
        else: 
            # keep non-integer values
            hchar *= number

    if(realize == True):
        # with a single sample, want strain in shape (M-1, Q-1, Z-1, F-1)
        hchar *= np.random.poisson(number)

    if(utils.isinteger(realize)):
        # with R realizations, 
        # to get strain in shape (M-1, Q-1, Z-1, F-1, R) we need
        # hchar in shape(M-1, Q-1, Z-1, F-1, 1)
        # Poisson sample in shape (1, 1, 1, 1, R)
        npois = np.random.poisson(number[...,np.newaxis], size = (number.shape + (realize,)))
        if(print_test):
            print('npois', npois.shape)
        hchar = hchar[...,np.newaxis] * npois


    if(print_test):
        print('hchar', hchar.shape, '\n', hchar)


    if(sum):
        # sum over all bins at a given frequency and realization
        hchar = np.sum(hchar, axis=(0, 1, 2))
        # NOTE I should check what big O time this is,  not sure
        if(print_test):
            print('hchar summed', hchar.shape, '\n', hchar)

    return np.sqrt(hchar)

In [ ]:
hc_bg, hc_ss, sspar, ssidx, maxhs, number_bg  = ss_gws_by_ndars(edges, number, realize=False, round=True, sum=True, ss=True)

# 2 Super Simple Example 

## 2.1 Build Model

In [ ]:
exname = 'Example 2'

### Choose the frequency bins at which to calculate the GWB
same as in semi-analytic-models.ipynb

In [ ]:
dur = 5.0*YR/3.1557600
cad = .5*YR/3.1557600
fobs = utils.nyquist_freqs(dur,cad)
fobs_edges = utils.nyquist_freqs_edges(dur,cad)
print(f"Number of frequency bins: {fobs.size-1}")
print(f"  between [{fobs[0]*YR:.2f}, {fobs[-1]*YR:.2f}] 1/yr")
print(f"          [{fobs[0]*1e9:.2f}, {fobs[-1]*1e9:.2f}] nHz")

### Build Semi-Analytic-Model with super simple parameters 

In [ ]:
exname = 'Example 2'

mtot=(1.0e6*MSOL/1.988409870698051, 1.0e8*MSOL/1.988409870698051, 3)
mrat=(1e-1, 1.0, 2)
redz=(1e-3, 1.0, 4)
sam = holo.sam.Semi_Analytic_Model(mtot, mrat, redz)
print('edges:', sam.edges)

### Get SAM edges and numbers as in sam.gwb()

In [ ]:
# get observed orbital frequency bin edges and centers 
# from observed GW frequency bin edges
# f_orb = f_GW/2
fobs_orb_edges = fobs_edges / 2.0
fobs_orb_cents = kale.utils.midpoints(fobs_edges) / 2.0

# dynamic_binary_number
# gets differential number of binaries per bin-vol per log freq interval
edges, dnum = sam.dynamic_binary_number(holo.hardening.Hard_GW, fobs_orb=fobs_orb_cents)
edges[-1] = fobs_orb_edges

# integrate (multiply by bin volume) within each bin
number = utils._integrate_grid_differential_number(edges, dnum, freq=False)
number = number * np.diff(np.log(fobs_edges))

## 2.2 Calculate strains

In [ ]:
# 1. Get GW Total Strain using gws_by_ndars()
# hc_tt = gws_by_ndars(edges, number, realize=False, round = True, sum=True)   
 
# cleaan
hc_bg, hc_ss, sspar, ssidx, maxhs, number_bg  = ss_gws_by_loops(edges, number, realize=False, round=True, sum=True, ss=True, print_test=True)

In [ ]:
hc_bg2, hc_ss2, sspar2, ssidx2, maxhs2, number_bg2 = ss_gws_by_loops_2(edges, number, realize=False, round=True, sum=True, ss=True, print_test=True)

In [ ]:
print(np.all(hc_ss == hc_ss2))
print(np.all(hc_bg == hc_bg2))
np.allclose(hc_bg, hc_bg2)
print(hc_ss - hc_ss2)

## 2.3 Test Plots

### BG, SS, and TT Strain

In [ ]:
# quick plot
fig, ax = plot.figax(xlabel='Frequency $f_\mathrm{obs}$ [1/yr]', 
                    ylabel='Characteristic Strain $h_c$', figsize=[10,4.5])
ax.set_title(exname)
xx = fobs * YR
print(len(hc_bg_ugly))

# plot a reference, pure power-law  strain spectrum:   h_c(f) = 1e-15 * (f * yr) ^ -2/3
yy = 1e-15 * np.power(xx, -2.0/3.0)
ax.plot(xx, yy, 'k--', alpha=0.25, lw=2.0, label = 'pure power law')

# total char strain (no sources subtracted)
ax.plot(xx, hc_tt, color='black', marker = 'x', lw=3, ls = 'dotted', alpha=1, label='total, by loops')

# gwb
ax.plot(xx, hc_bg_ugly, color='lightblue', marker = 'o', lw=3, ms=12,
        ls = 'dotted', alpha=.5, label='background, ugly loops')
# loudest source per bin
ax.scatter(xx, hc_ss_ugly, color='lightblue', marker = 'o', s=100,
           edgecolor='k', alpha=.5, label='single source, ugly loops')


# gwb
ax.plot(xx, hc_bg, color='b', marker = 'o', lw=3, ls = 'dotted', alpha=.5, label='background, pretty loops')
# loudest source per bin
ax.scatter(xx, hc_ss, color='b', marker = 'o', edgecolor='k', alpha=.5, label='single source, pretty loops')


# ax.plot(xx, hc_bg+hc_ss, color='r', label='bg + ss', alpha=.5)
ax.plot(xx, np.sqrt(hc_bg**2+hc_ss**2), color='r', label=r'$sqrt (bg^2 + ss^2)$', alpha=.5)

legend_gwb = ax.legend(bbox_to_anchor=(.1,.15), bbox_transform=fig.transFigure, loc='lower left')


# ax.set_ylim(1e-16, 3e-15)
fig.tight_layout()

##### number - number_bg check

In [ ]:
# plot number vs anything at 1 frequency 
freqs_in_nHz = fobs*10**9
# plot number - single sources vs anything at 1 frequency,
# should only vary in one cell
fig, ax = plt.subplots(1,len(fobs), figsize = (10,3), sharey=True)
fig.suptitle('number - number_bg for each bin, '+ exname)
ax[0].set_ylabel('number - number_bg')
bins = np.arange(0, number[...,0].size, 1)
bins = np.reshape(bins, number[...,0].shape)
print(bins.shape)
print(number[...,0].shape)
for f in range(len(fobs)):
    ax[f].scatter(bins, (number[...,f] - number_bg[...,f]))
    ax[f].set_title('$f_\mathrm{obs}$ = %dnHz' % freqs_in_nHz[f])
    ax[f].set_xlabel('bin')
fig.tight_layout()


# 3 Slightly More Complex Example
Let's see what windows can handle. Limiting factor seems to be the total mass upper limit.

In [ ]:
exname = 'Example 3'

## 3.1 Build Model
### Choose the frequency bins at which to calculate the GWB
same as in semi-analytic-models.ipynb

In [ ]:
dur = 5.0*YR/3.1557600
cad = .5*YR/3.1557600
fobs = utils.nyquist_freqs(dur,cad)
fobs_edges = utils.nyquist_freqs_edges(dur,cad)
print(f"Number of frequency bins: {fobs.size-1}")
print(f"  between [{fobs[0]*YR:.2f}, {fobs[-1]*YR:.2f}] 1/yr")
print(f"          [{fobs[0]*1e9:.2f}, {fobs[-1]*1e9:.2f}] nHz")

### Build Semi-Analytic-Model with super simple parameters 

In [ ]:
mtot=(1.0e6*MSOL/1.988409870698051, 4.0e9*MSOL, 25)
mrat=(1e-1, 1.0, 25)
redz=(1e-3, 10.0, 25)
sam = holo.sam.Semi_Analytic_Model(mtot, mrat, redz)
print('edges:', sam.edges)

### Get SAM edges and numbers as in sam.gwb()

In [ ]:
# get observed orbital frequency bin edges and centers 
# from observed GW frequency bin edges
# f_orb = f_GW/2
fobs_orb_edges = fobs_edges / 2.0
fobs_orb_cents = kale.utils.midpoints(fobs_edges) / 2.0

# dynamic_binary_number
# gets differential number of binaries per bin-vol per log freq interval
edges, dnum = sam.dynamic_binary_number(holo.hardening.Hard_GW, fobs_orb=fobs_orb_cents)
edges[-1] = fobs_orb_edges

# integrate (multiply by bin volume) within each bin
number = utils._integrate_grid_differential_number(edges, dnum, freq=False)
number = number * np.diff(np.log(fobs_edges))

## 3.2 Calculate strains

In [ ]:
# 1. Get GW Total Strain using gws_by_ndars()
# hc_tt = ss_gws_by_loops(edges, number, realize=False, round = True, sum=True, ss=False)   
hc_tt = gws_by_ndars(edges, number, realize=False, round = True, sum=True)  
 
# 2 Get bg (background) and ss (single source) strains by ss_gws_by_loops
hc_bg, hc_ss, sspar, ssidx, maxhs, number_bg = ss_gws_by_loops(edges, number, realize=False, round=True, sum=True, ss=True)

In [ ]:
print(np.all(np.abs(number_bg-number_bg3)<=2))

### figuring out loops2 method

In [ ]:
hc_bg2, hc_ss2, sspar2, ssidx2, maxhs2, number_bg2 = ss_gws_by_loops_2(edges, number, realize=False, round=True, sum=True, ss=True)

In [ ]:
print(hc_bg2)

In [ ]:
plt.scatter(np.arange(0, number_bg.size, 1), (number_bg - number_bg2).flatten(), s=.5)

In [ ]:
print(number_bg[np.where(number_bg%1 != 0)])

In [ ]:
unmatched = np.where(number_bg-number_bg2 >0)
print(number_bg2[unmatched])

In [ ]:
print(np.all(hc_bg == hc_bg2))
print(np.all(hc_ss == hc_ss2))
print(np.all(sspar == sspar2))
print(np.all(ssidx == ssidx2))
print(np.all(number_bg == number_bg2))

In [ ]:
np.isclose(hc_ss2, hc_ss)

In [ ]:
test = (hc_bg2**2 + hc_ss2**2)
error = (test-hc_tt**2)/hc_tt**2
# np.isclose(hc_tt, )
print(error)
print(hc_tt - np.sqrt(hc_ss2**2 + hc_bg2**2))

## 3.3 Test Plots

### BG, SS, and TT Strain

In [ ]:
# quick plot
fig, ax = plot.figax(xlabel='Frequency $f_\mathrm{obs}$ [1/yr]', 
                    ylabel='Characteristic Strain $h_c$', figsize=[10,4.5])
ax.set_title(exname)
xx = fobs * YR
print(fobs.shape)
print(len(hc_tt))
# plot a reference, pure power-law  strain spectrum:   h_c(f) = 1e-15 * (f * yr) ^ -2/3
yy = 1e-15 * np.power(xx, -2.0/3.0)
ax.plot(xx, yy, 'k--', alpha=0.25, lw=2.0, label = 'pure power law')

# total char strain (no sources subtracted)
ax.plot(xx, hc_tt, color='black', marker = 'x', lw=3, ls = 'dotted', alpha=1, label='total, by loops')


# gwb
ax.plot(xx, hc_bg, color='b', marker = 'o', lw=3, ls = 'dotted', alpha=.5, label='background, pretty loops')
# loudest source per bin
ax.scatter(xx, hc_ss, color='b', marker = 'o', edgecolor='k', alpha=.5, label='single source, pretty loops')


# ax.plot(xx, hc_bg+hc_ss, color='r', label='bg + ss', alpha=.5)
ax.plot(xx, np.sqrt(hc_bg**2+hc_ss**2), color='r', label=r'$sqrt (bg^2 + ss^2)$', alpha=.5)

legend_gwb = ax.legend(bbox_to_anchor=(.1,.15), bbox_transform=fig.transFigure, loc='lower left')


# ax.set_ylim(1e-16, 3e-15)
fig.tight_layout()

##### number - number_bg check

In [ ]:
# plot number vs anything at 1 frequency 
freqs_in_nHz = fobs*10**9
# plot number - single sources vs anything at 1 frequency,
# should only vary in one cell
fig, ax = plt.subplots(1,len(fobs), figsize = (10,3), sharey=True)
fig.suptitle('number - number_bg for each bin, '+ exname)
ax[0].set_ylabel('number - number_bg')
bins = np.arange(0, number[...,0].size, 1)
bins = np.reshape(bins, number[...,0].shape)
print(bins.shape)
print(number[...,0].shape)
for f in range(len(fobs)):
    ax[f].scatter(bins, (number[...,f] - number_bg[...,f]))
    ax[f].set_title('$f_\mathrm{obs}$ = %dnHz' % freqs_in_nHz[f])
    ax[f].set_xlabel('bin')
fig.tight_layout()


# 4. Even More Complex Example - Casting 64bit


In [ ]:
exname = 'Example 4'

## 4.1 Calculate Strains
### Choose the frequency bins at which to calculate the GWB

In [ ]:
dur = 5.0*YR/3.1557600
cad = .2*YR/3.1557600
fobs = utils.nyquist_freqs(dur,cad)
fobs_edges = utils.nyquist_freqs_edges(dur,cad)
print(f"Number of frequency bins: {fobs.size-1}")
print(f"  between [{fobs[0]*YR:.2f}, {fobs[-1]*YR:.2f}] 1/yr")
print(f"          [{fobs[0]*1e9:.2f}, {fobs[-1]*1e9:.2f}] nHz")

### Build Semi-Analytic-Model with super simple parameters 

In [ ]:
mtot=(1.0e6*MSOL/1.988409870698051, (4.0e11*MSOL).astype(np.float64), 25)
mrat=(1e-1, 1.0, 25)
redz=(1e-3, 10.0, 25)
sam = holo.sam.Semi_Analytic_Model(mtot, mrat, redz)
print('edges:', sam.edges)

### Get SAM edges and numbers as in sam.gwb()

In [ ]:
# get observed orbital frequency bin edges and centers 
# from observed GW frequency bin edges
# f_orb = f_GW/2
fobs_orb_edges = fobs_edges / 2.0
fobs_orb_cents = kale.utils.midpoints(fobs_edges) / 2.0

# dynamic_binary_number
# gets differential number of binaries per bin-vol per log freq interval
edges, dnum = sam.dynamic_binary_number(holo.hardening.Hard_GW, fobs_orb=fobs_orb_cents)
edges[-1] = fobs_orb_edges

# integrate (multiply by bin volume) within each bin
number = utils._integrate_grid_differential_number(edges, dnum, freq=False)
number = number * np.diff(np.log(fobs_edges))

### Calculate strains

In [ ]:
# 1. Get GW Total Strain using gws_by_ndars()
# hc_tt = gws_by_loops(edges, number, realize=False, round = True, sum=True)   
 
# 2 Get bg (background) and ss (single source) strains by ss_gws_by_loops
hc_bg, hc_ss, sspar, ssidx, maxhs, number_bg = ss_gws_by_loops(edges, number, realize=False, round=True, sum=True, ss=True)

In [ ]:
hc_bg2, hc_ss2, sspar2, ssidx2, maxhs2, number_bg2 = ss_gws_by_loops_2(edges, number, realize=False, round=True, sum=True, ss=True)

In [ ]:
print(np.all(hc_ss == hc_ss2))
print(np.all(hc_bg == hc_bg2))
np.allclose(hc_bg, hc_bg2)
print((hc_ss - hc_ss2)/hc_ss)

## 4.3 Test Plots

### BG, SS, and TT Strain

In [ ]:
# quick plot
fig, ax = plot.figax(xlabel='Frequency $f_\mathrm{obs}$ [1/yr]', 
                    ylabel='Characteristic Strain $h_c$', figsize=[10,4.5])
ax.set_title(exname)
xx = fobs * YR

# plot a reference, pure power-law  strain spectrum:   h_c(f) = 1e-15 * (f * yr) ^ -2/3
yy = 1e-15 * np.power(xx, -2.0/3.0)
ax.plot(xx, yy, 'k--', alpha=0.25, lw=2.0, label = 'pure power law')

# total char strain (no sources subtracted)
ax.plot(xx, hc_tt, color='black', marker = 'x', lw=3, ls = 'dotted', alpha=1, label='total, by loops')


# gwb
ax.plot(xx, hc_bg, color='b', marker = '.', lw=3, ls = 'dotted', alpha=.8, label='background, by loops')
# loudest source per bin
ax.scatter(xx, hc_ss, color='b', marker = 'o', edgecolor='k', alpha=.8, label='single source, by loops')


# ax.plot(xx, hc_bg+hc_ss, color='r', label='bg + ss')
ax.plot(xx, np.sqrt(hc_bg**2+hc_ss**2), color='r', label=r'$sqrt (bg^2 + ss^2)$', alpha=.5)


legend_gwb = ax.legend(bbox_to_anchor=(.1,.15), bbox_transform=fig.transFigure, loc='lower left')


# ax.set_ylim(1e-16, 3e-15)
fig.tight_layout()

##### number - number_bg check

In [ ]:
# plot number vs anything at 1 frequency 
freqs_in_nHz = fobs*10**9
# plot number - single sources vs anything at 1 frequency,
# should only vary in one cell
fig, ax = plt.subplots(1,len(fobs), figsize = (15,3), sharey=True)
fig.suptitle('number - number_bg for each bin, '+ exname)
ax[0].set_ylabel('number - number_bg')
bins = np.arange(0, number[...,0].size, 1)
bins = np.reshape(bins, number[...,0].shape)
print(bins.shape)
print(number[...,0].shape)
for f in range(len(fobs)):
    ax[f].scatter(bins, (number[...,f] - number_bg[...,f]))
    ax[f].set_title('$f_\mathrm{obs}$ = %dnHz' % freqs_in_nHz[f])
    ax[f].set_xlabel('bin')
fig.tight_layout()


In [ ]:
test = (hc_bg**2 + hc_ss**2)
error = (test-hc_tt**2)/hc_tt**2
# np.isclose(hc_tt, )
print(error)
print(hc_tt - np.sqrt(hc_ss**2 + hc_bg**2))

use n>0